In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
ds = xr.open_dataset("../../NC/compare.nc")  

In [ ]:
def get_diff_quar(ds, name, time):
    global diff_v
    # 构造变量名
    var_basin  = f"basin_{name}"
    var_region = f"region_{name}"

    # 取该年数据
    da_basin  = ds[var_basin].sel(time=time)
    da_region = ds[var_region].sel(time=time)

    # 差值
    diff = da_region - da_basin

    # 有效区域 mask（至少一个 > 0）
    mask = (da_basin > 0) | (da_region > 0)

    # 应用 mask → 无效格点变为 NaN
    diff_v = diff.where(mask)
    vals = diff_v.values.flatten()
    vals = vals[~np.isnan(vals)]

    Q1     = np.percentile(vals, 25)
    Median = np.percentile(vals, 50)
    Q3     = np.percentile(vals, 75)
    IQR    = Q3 - Q1

    return Q1,Median,Q3,IQR

In [ ]:
years = list(range(2010, 2101, 10))   # 2010,2020,...,2100
years.insert(0, 2005)
year = "2100"
names = ["agri","forest","grassland"]
time =f"{year}-01-01"

for name in names:
    Q1_list, Median_list, Q3_list, IQR_list = [], [], [], []
    for i in years:
        time =f"{i}-01-01"
        Q1,Median,Q3,IQR=get_diff_quar(ds, name, time)
        Q1_list.append(Q1)
        Median_list.append(Median)
        Q3_list.append(Q3)
        IQR_list.append(IQR)

    plt.figure(figsize=(8,5))
    plt.plot(years, Q1_list, marker='o',label="Q1")
    plt.plot(years, Median_list, marker='s',label="Median")
    plt.plot(years, Q3_list, marker='x',label="Q3")
    plt.plot(years, IQR_list, marker='+',label="IQR")
    plt.xlabel("Year")
    plt.ylabel("Q1 (25th percentile)")
    plt.title(f"Quartile of diff ({name}) over years")
    plt.grid(True)
    plt.legend()
    plt.savefig(f"../../plot/analysis/{name}_quar.png", dpi=300, bbox_inches='tight')
    plt.show()

